# 1.데이터 수집- OpenAPI 사용하기

In [3]:
import requests
import pandas as pd
import json

# 1. 타겟 사이트 url
url = "http://data4library.kr/api/loanItemSrch?format=json&startDt=2023-01-01&endDt=2023-12-30&age=20&authKey=f4666840f0e55ae1169557f6bf1054d2c782bfa99ba78522c41aa9bb3d79686c"
res = requests.get(url)

# 2. 페이지 로딩
if res.status_code == 200:
    print("Request successful")
else:
    print("Error:", res.status_code)
    # exit()  

# 3. Decode binary content to UTF-8 string
content_str = res.content.decode('utf-8')

# 4. Parse the JSON data
data = json.loads(content_str)

# Print the parsed JSON data
print(data)

# 5. ETL (Extract and save it as list)

books = []
# for d in data['response']['docs']:
#     books.append(d['doc'])

books = [d['doc'] for d in data['response']['docs']]  # list complrehension

# 6. 리스트를 이용하여 데이터 프레임 생성 : list -> DataFrame
books_df = pd.DataFrame(books)

print(books_df)

books_df.to_json('20s_best_book_2023.json')



Request successful
{'response': {'error': 'API 활성화 상태가아닙니다.'}}


KeyError: 'docs'

## 2. 검색 결과 페이지 가져오기 - "20대가 가장 좋아하는 도서목록"

In [ ]:
# json file 로딩 -> dataFrame 객체 변환
books_df = pd.read_json('20s_best_book_2023.json')
books_df

In [ ]:
# 특정 열을 선택하여 새 데이터프레임 생성 - 컬럼 인덱싱 이용
books_df.columns
books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']]
books.head()

In [ ]:
# 원하는 데이터프레임 행과 열 동시에 선택하기 - df.loc[]
books_df.loc[[0,1], ['bookname','authors']]

In [ ]:
# 원하는 데이터프레임 행과 열 동시에 슬라이싱으로 선택하기 - df.loc[:,:]
books = books_df.loc[:, 'no':'isbn13']
books.head()

## 3. HTML에서 데이터 추출하기 - BeautifulSoup
    1. 크롬 브라우저 개발자 도구로 HTML 태그 찾기
    2. 태그 위치 찾기 : soup.find('찾을 태그 이름',attrs={'태그 속성을 딕셔너리로 지정'} 
    3. soup.select()
    

### [문제] BeautifulSoup으로 HTML에서 첫 번째 도서명의 링크 주소 찾기
* 타겟 웹페이지 : https://www.yes24.com/Product/Search?query=9791189327156 # '물고기는 존재하지 않는다'의 ISBN

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
isbn = 9791189327156
#isbn = 74261416
url = 'https://www.yes24.com/Product/Search?query={}'
r = requests.get(url.format(isbn))
soup = BeautifulSoup(r.text, 'html.parser')
prd_link = soup.find('a', attrs={'class':'gd_name'})
print(prd_link)
print(prd_link['href'])

### [문제] BeautifulSoup으로 HTML에서 첫 번째 도서의 페이지수가 담긴 HTML의 위치 찾기
* 타겟 웹페이지 : https://www.yes24.com/Product/Goods/105526047
* soup.select()

In [ ]:
# soup.select()
url = 'https://www.yes24.com/Product/Goods/105526047'
r= requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td')
print(prd_detail)
if prd_detail is not None:
    page = prd_detail[0].text.split('|')[0]
    print(page)
else:
    print("The element you're trying to select doesn't exist.")


### [문제]  20대가 선호하는 전체 도서의 쪽수 구하기 

In [ ]:
#1. 사용자 정의 함수
def get_page_cnt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'https://www.yes24.com/Product/Search?query={}'
    # URL에 ISBN을 넣어 HTML 가져오기.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   
    # 검색 결과에서 해당 도서를 선택하기.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져오기.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 쪽수가 들어 있는 td를 찾기.
    prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td')
    # print(prd_detail)
    result = prd_detail[0]
    page = result.text.split('|')[0]
    return page
    

In [ ]:
get_page_cnt(9791189327156)

In [ ]:
top10_books = books.head(10) # 인기 도서 10권 추출
top10_books

In [ ]:
# 10개 도서의 쪽수를 한번에 구하기
def get_page_cnt2(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

In [ ]:
# 각 행의 반복 작업을 수행하기 위해 데이터프레임의 apply()메서드 적용
page_count = top10_books.apply(get_page_cnt2, axis = 1)
print(page_count)

### [문제]  top10_books와 쪽수 시리즈 합치기 : merge()

In [ ]:
# 쪽수 데이터프레임 생성
type(page_count)
page_count.name = 'page_count'
print(page_count)

In [ ]:
# 데이터프레임과 시리즈 합치기
top10_with_page_count = pd.merge(top10_books, page_count,
                                 left_index=True, right_index=True)
top10_with_page_count

## [연습] merge() 함수의 매개변수 이해하기

In [ ]:
df1 = pd.DataFrame({'col1': ['a','b','c'], 'col2': [1,2,3]})
df1

In [ ]:
df2 = pd.DataFrame({'col1': ['a','b','d'], 'col3': [10,20,30]})
df2

In [ ]:
#df1.merge(df2, on = 'col1')

In [ ]:
pd.merge(df1, df2, on='col1')


In [ ]:
pd.merge(df1, df2, how='left', on='col1')

In [ ]:
pd.merge(df1, df2, how='right', on='col1')

In [ ]:
pd.merge(df1, df2, how='outer', on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col1', right_on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col2', right_index=True)